<a href="https://colab.research.google.com/github/petroDavydov/goit-DeepLearningForComputerVisionAndNLP/blob/main/eda_pinguinius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dataclasses import dataclass

import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# filter warnings
warnings.filterwarnings('ignore')


Завантажимо набір даних і попередньо переглянемо:



In [ ]:
df = pd.read_csv('/content/Module_2_Lecture_2_Class_penguins.csv')

df.sample(5, random_state=42)


Виведемо базову інформацію про набір даних.

In [ ]:
df.info()


З колонки Non-Null Count бачимо, що тільки декілька рядків даних мають пропущені значення. Можемо видалити їх з датасету.

In [ ]:
df = df.dropna().reset_index(drop=True)
df

Подивимось на розподіл цільової змінної.

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.countplot(data=df, x='species', palette='husl')
for i in ax.containers:
    ax.bar_label(i)
    ax.set_xlabel("value")
    ax.set_ylabel("count")

plt.suptitle("Target feature distribution")

plt.tight_layout()
plt.show()


# ***💡 У випадках, як цей, коли класи не сильно дисбалансовані, підходящою метрикою є аccuracy. Оскільки точність вимірює частку правильно класифікованих прикладів у загальній їх кількості, використання цієї метрики є доцільним для оцінки ефективності цієї моделі.***

# Подивимось розподіл категоріальної змінної ***island.***

In [ ]:
plt.figure(figsize=(4,3))
ax = sns.countplot(data=df, x='island', palette = 'husl')
for i in ax.containers:
    ax.bar_label(i)
    ax.set_xlabel("value")
    ax.set_ylabel("count")

plt.suptitle("Island feature distribution")

plt.tight_layout()
plt.show()

# ***💡 Дисбаланс класів цієї змінної може вказувати на дисбаланс у розподілі даних у реальному житті. У цьому випадку необхідно бути обережними, щоб не видаляти записи вхідних даних або надлишково їх дискретизувати (oversample), оскільки це може внести зміщення в нашу модель.***

# **Подивимось на попарний розподіл числових ознак.**

In [ ]:
plt.figure(figsize=(5,5))
sns.pairplot(data=df, hue='species').fig.suptitle('Numeric features distribution', y=1)
plt.show()


# ***Підготовка ознак моделі***

Для моделювання цієї задачі залишимо тільки числові змінні.

In [ ]:
features = ['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

df = df.loc[:, features]

df

💡 В якості “завдання із зірочкою” можете залишити усі ознаки, що є в наборі даних, та опрацювати їх таким чином, щоб замість категоріальних змінних мати числові.

Перетворимо категоріальну таргетну змінну в числову.



In [ ]:
df.loc[df['species']=='Adelie', 'species']=0
df.loc[df['species']=='Gentoo', 'species']=1
df.loc[df['species']=='Chinstrap', 'species']=2
df = df.apply(pd.to_numeric)

df.head(2)


Представимо матрицю ознак
X та вектор таргетової змінної
y як numpy-масив.

In [ ]:
X = df.drop('species', axis =1).values
y = df['species'].values

X
y

Щоб гарантувати, що ознаки будуть представлені в одному масштабі, використаємо StandardScaler.

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X


Розділимо дані на тренувальні та тестові.

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X, y, random_state = 42, test_size =0.33, stratify=y)

X_train
X_test
y_train
y_test

Для подальшої роботи з інструментами фреймворку PyTorch перетворимо дані з numpy-масивів у torch.tensor.

In [ ]:
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).long()

X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).long()

print(X_train[:1])
print(y_train[:10])


"-----------------------------------------"

# ***Задача багатокласової класифікації. Моделювання та аналіз результатів***



Підготовка моделі

Створимо клас нейронної мережі для вирішення задачі багатокласової класифікації.

In [ ]:
class LinearModel(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim=20, out_dim=3):
        super().__init__()

        self.features = torch.nn.Sequential(

            nn.Linear(in_dim, hidden_dim),
            torch.nn.ReLU(),

            nn.Linear(hidden_dim, out_dim),
            nn.Softmax()
        )

    def forward(self, x):
        output = self.features(x)
        return output


In [ ]:
# # Second variant

# class LinearModel(torch.nn.Module):
#     def __init__(self, in_dim, hidden_dim=20, out_dim=3):
#         super().__init__()

#         self.features = torch.nn.Sequential(
#             nn.Linear(in_dim, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, out_dim)   # без Softmax
#         )

#     def forward(self, x):
#         return self.features(x)


***Ініціалізуємо модель.***

In [ ]:
model = LinearModel(X_train.shape[1], 20, 3)


In [ ]:
criterion = nn.CrossEntropyLoss()


В якості оптимізаційного алгоритму використаємо Stochastic Gradient Descent (SGD).

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


Кількість епох визначимо рівною 400.

In [ ]:
num_epoch = 350


Створимо декілька об’єктів-списків, у які будемо зберігати результати тренування: accuracy та loss.

In [ ]:
train_loss = []
test_loss = []

train_accs = []
test_accs = []


# ***Тренування***

In [ ]:
# model.train()
# outputs = model(X_train)
# loss = criterion(outputs, y_train)
# train_loss.append(loss.cpu().detach().numpy())
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()
# acc = 100 * torch.sum(y_train==torch.max(outputs.data, 1)[1]).double() / len(y_train)
# train_accs.append(acc)


# ***Тестування***

In [ ]:
model.eval()


***Робимо forward pass та розраховуємо функцію втрат, але не робимо крок оптимізації. Розраховуємо точність на тестових даних.***

In [ ]:
# outputs = model(X_test)

# loss = criterion(outputs, y_test)
# test_loss.append(loss.cpu().detach().numpy())

# acc = 100 * torch.sum(y_test==torch.max(outputs.data, 1)[1]).double() / len(y_test)
# test_accs.append(acc)


# ***Отже, маємо наступний цикл, в якому тренуємо і тестуємо нашу модель.***

In [ ]:
for epoch in range(num_epoch):

    # train the model
    model.train()

    outputs = model(X_train)

    loss = criterion(outputs, y_train)
    train_loss.append(loss.cpu().detach().numpy())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = 100 * torch.sum(y_train==torch.max(outputs.data, 1)[1]).double() / len(y_train)
    train_accs.append(acc)

    if (epoch+1) % 10 == 0:
        print ('Epoch [%d/%d] Loss: %.4f   Acc: %.4f'
                       %(epoch+1, num_epoch, loss.item(), acc.item()))

    # test the model

    model.eval()
    with torch.no_grad():
        outputs = model(X_test)

        loss = criterion(outputs, y_test)
        test_loss.append(loss.cpu().detach().numpy())

        acc = 100 * torch.sum(y_test==torch.max(outputs.data, 1)[1]).double() / len(y_test)
        test_accs.append(acc)


# ***Аналіз результатів***

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(train_loss, label='Train')
plt.plot(test_loss, label='Validation')
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Cross-Entropy Loss')
plt.title('Training vs Validation Loss')
plt.show()


In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(train_accs, label='Train')
plt.plot(test_accs, label='Validation')
plt.legend(loc='best')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Metric')
plt.show()
